In [4]:
import sys
print(sys.version)

3.7.0 (v3.7.0:1bf9cc5093, Jun 26 2018, 23:26:24) 
[Clang 6.0 (clang-600.0.57)]


In [5]:
# Not necessary at all, but to demonstrate that I'm aware that BeautifulSoup4 must be installed
!{sys.executable} -m pip install --upgrade pip
!{sys.executable} -m pip install BeautifulSoup4

Requirement already up-to-date: pip in /Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages (18.1)


In [47]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import json
import csv
import time
import datetime

In [20]:
def perform_http_get(url):
    """
    Note:
        This is a function that tries to connect to a given URL, if it fails, a retry will occurs in 3 sec.  

    Args:
        url (str): A string representing the URL we want to integrate.

    Returns:
        The http response content 
    """

    r = requests.get(url)
    print('Status Code: %s, page encoding: %s' % (r.status_code, r.encoding))
    return BeautifulSoup(r.content, 'html.parser')
        

In [70]:
main_url = 'https://en.wikipedia.org/wiki/List_of_UFC_events'
soup = perform_http_get(main_url)
table_past_events = soup.find('table', {'id': 'Past_events'})

rows = table_past_events.findAll('tr')
type(rows)

for row in rows:
    cells = row.findAll('td')
    if len(cells) > 0 :
        links = cells[1].findAll('a')
        for link in links:
            
            secondary_url = 'https://en.wikipedia.org' + link.get('href')
            individual_event = perform_http_get(secondary_url)
            
            table = individual_event.find("table",{"class":"toccolours"})
            if table is None:
                print("NOOOOO")
            
            #d = dict([(, link.contents[0])])
            # detail = { 'link' : link.get('href'), 'name' : link.contents[0]}
            #Events.append(d)

Status Code: 200, page encoding: UTF-8
Status Code: 200, page encoding: UTF-8
Status Code: 200, page encoding: UTF-8
Status Code: 200, page encoding: UTF-8
Status Code: 200, page encoding: UTF-8
Status Code: 200, page encoding: UTF-8
Status Code: 200, page encoding: UTF-8
Status Code: 200, page encoding: UTF-8
Status Code: 200, page encoding: UTF-8
Status Code: 200, page encoding: UTF-8
Status Code: 200, page encoding: UTF-8
Status Code: 200, page encoding: UTF-8
Status Code: 200, page encoding: UTF-8
Status Code: 200, page encoding: UTF-8
Status Code: 200, page encoding: UTF-8
Status Code: 200, page encoding: UTF-8
Status Code: 200, page encoding: UTF-8
Status Code: 200, page encoding: UTF-8
Status Code: 200, page encoding: UTF-8
Status Code: 200, page encoding: UTF-8
Status Code: 200, page encoding: UTF-8
Status Code: 200, page encoding: UTF-8
Status Code: 200, page encoding: UTF-8
Status Code: 200, page encoding: UTF-8
Status Code: 200, page encoding: UTF-8
Status Code: 200, page en

Status Code: 200, page encoding: UTF-8
Status Code: 200, page encoding: UTF-8
Status Code: 200, page encoding: UTF-8
Status Code: 200, page encoding: UTF-8
Status Code: 200, page encoding: UTF-8
Status Code: 200, page encoding: UTF-8
Status Code: 200, page encoding: UTF-8
Status Code: 200, page encoding: UTF-8
Status Code: 200, page encoding: UTF-8
Status Code: 200, page encoding: UTF-8
Status Code: 200, page encoding: UTF-8
Status Code: 200, page encoding: UTF-8
Status Code: 200, page encoding: UTF-8
Status Code: 200, page encoding: UTF-8
Status Code: 200, page encoding: UTF-8
Status Code: 200, page encoding: UTF-8
Status Code: 200, page encoding: UTF-8
Status Code: 200, page encoding: UTF-8
Status Code: 200, page encoding: UTF-8
Status Code: 200, page encoding: UTF-8
Status Code: 200, page encoding: UTF-8
Status Code: 200, page encoding: UTF-8
Status Code: 200, page encoding: UTF-8
Status Code: 200, page encoding: UTF-8
Status Code: 200, page encoding: UTF-8
Status Code: 200, page en

KeyboardInterrupt: 

In [66]:
df = pd.DataFrame(Events)
df['Event'] = Events

df

,/wiki/2012_in_UFC#UFC_149:_Faber_vs._Barão,/wiki/Ortiz_vs._Shamrock_3:_The_Final_Chapter,/wiki/The_Ultimate_Fighter:_A_Champion_Will_Be_Crowned#The_Ultimate_Fighter_20_Finale,/wiki/The_Ultimate_Fighter:_A_New_World_Champion#The_Ultimate_Fighter_26_Finale,/wiki/The_Ultimate_Fighter:_American_Top_Team_vs._Blackzilians#The_Ultimate_Fighter_21_Finale,/wiki/The_Ultimate_Fighter:_Brazil_2#The_Ultimate_Fighter:_Brazil_2_Finale,/wiki/The_Ultimate_Fighter:_Brazil_3#The_Ultimate_Fighter_Brazil_3_Finale:_Miocic_vs._Maldonado,/wiki/The_Ultimate_Fighter:_China#The_Ultimate_Fighter_China_Finale:_Kim_vs._Hathaway,/wiki/The_Ultimate_Fighter:_Heavyweights_Finale,/wiki/The_Ultimate_Fighter:_Latin_America_2#The_Ultimate_Fighter_Latin_America_2_Finale:_Magny_vs._Gastelum,...,/wiki/UFC_on_Fuel_TV:_Gustafsson_vs._Silva,/wiki/UFC_on_Fuel_TV:_Korean_Zombie_vs._Poirier,/wiki/UFC_on_Fuel_TV:_Mousasi_vs._Latifi,/wiki/UFC_on_Fuel_TV:_Munoz_vs._Weidman,/wiki/UFC_on_Fuel_TV:_Sanchez_vs._Ellenberger,/wiki/UFC_on_Fuel_TV:_Silva_vs._Stann,/wiki/UFC_on_Fuel_TV:_Struve_vs._Miocic,/wiki/Ultimate_Ultimate_1995,/wiki/Ultimate_Ultimate_1996,Event
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,{'/wiki/UFC_229': UFC 229: Khabib vs. McGregor}
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,{'/wiki/UFC_Fight_Night:_Santos_vs._Anders': U...
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,{'/wiki/UFC_Fight_Night:_Hunt_vs._Oliynyk': UF...
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,{'/wiki/UFC_228': UFC 228: Woodley vs. Till}
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,{'/wiki/UFC_Fight_Night:_Gaethje_vs._Vick': UF...
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,{'/wiki/UFC_227': UFC 227: Dillashaw vs. Garbr...
6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,{'/wiki/UFC_on_Fox:_Alvarez_vs._Poirier_2': UF...
7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,{'/wiki/UFC_Fight_Night:_Shogun_vs._Smith': UF...
8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,{'/wiki/UFC_Fight_Night:_dos_Santos_vs._Ivanov...
9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,{'/wiki/UFC_226': UFC 226: Miocic vs. Cormier}
